In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

2022-10-05 18:43:02.009287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop("EIN",1)
application_df = application_df.drop("NAME",1)
application_df.head()

/var/folders/3y/sg53g9yx2l152vz_g8dwv8p00000gn/T/ipykernel_5772/1729653226.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop("EIN",1)
/var/folders/3y/sg53g9yx2l152vz_g8dwv8p00000gn/T/ipykernel_5772/1729653226.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop("NAME",1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 200].index)


# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1800].index)


# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Generate our categorical variable lists
application_cat = ["APPLICATION_TYPE",
                    "AFFILIATION",            
                    "CLASSIFICATION",         
                    "USE_CASE",             
                    "ORGANIZATION",                               
                    "INCOME_AMT",          
                    "SPECIAL_CONSIDERATIONS"]

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/Users/johannahdelunas/opt/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
df = application_df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(["APPLICATION_TYPE",
                    "AFFILIATION",            
                    "CLASSIFICATION",         
                    "USE_CASE",             
                    "ORGANIZATION",                               
                    "INCOME_AMT",          
                    "SPECIAL_CONSIDERATIONS"],1)
df.head()

/var/folders/3y/sg53g9yx2l152vz_g8dwv8p00000gn/T/ipykernel_5772/3370801497.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(["APPLICATION_TYPE",


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = df["IS_SUCCESSFUL"].values
X = df.drop(["IS_SUCCESSFUL"],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

/var/folders/3y/sg53g9yx2l152vz_g8dwv8p00000gn/T/ipykernel_5772/4058077970.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop(["IS_SUCCESSFUL"],1)


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Adding more neurons to a hidden layer

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               4400      
                                                                 
 dense_7 (Dense)             (None, 30)                3030      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7,461
Trainable params: 7,461
Non-trainable params: 0
_________________________________________________________________


In [23]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("Checkpoints/",exist_ok=True)
Checkpoint_path = "Checkpoints/weights.{epoch:02d}.hdf5"
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=Checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch", period=5)

In [24]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 718us/step - loss: 52722.5703 - accuracy: 0.4912
Epoch 2/100
804/804 [==============================] - 1s 741us/step - loss: 74150.0938 - accuracy: 0.5092
Epoch 3/100
804/804 [==============================] - 1s 711us/step - loss: 23510.3613 - accuracy: 0.4954
Epoch 4/100
804/804 [==============================] - 1s 703us/step - loss: 47310.8945 - accuracy: 0.4984
Epoch 5/100
778/804 [============================>.] - ETA: 0s - loss: 39754.6445 - accuracy: 0.4940
Epoch 5: saving model to Checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 721us/step - loss: 38480.3477 - accuracy: 0.4946
Epoch 6/100
804/804 [==============================] - 1s 708us/step - loss: 5456.5454 - accuracy: 0.5095
Epoch 7/100
804/804 [==============================] - 1s 712us/step - loss: 6214.9023 - accuracy: 0.5082
Epoch 8/100
804/804 [==============================] - 1s 727us/step - loss: 9402.0625 - accuracy: 0.4993
Epoch

804/804 [==============================] - 1s 700us/step - loss: 0.6911 - accuracy: 0.5335
Epoch 64/100
804/804 [==============================] - 1s 702us/step - loss: 0.6911 - accuracy: 0.5335
Epoch 65/100
787/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5336
Epoch 65: saving model to Checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 712us/step - loss: 0.6911 - accuracy: 0.5335
Epoch 66/100
804/804 [==============================] - 1s 716us/step - loss: 0.6911 - accuracy: 0.5335
Epoch 67/100
804/804 [==============================] - 1s 709us/step - loss: 0.6910 - accuracy: 0.5335
Epoch 68/100
804/804 [==============================] - 1s 704us/step - loss: 0.6911 - accuracy: 0.5335
Epoch 69/100
804/804 [==============================] - 1s 706us/step - loss: 0.6912 - accuracy: 0.5335
Epoch 70/100
777/804 [===========================>..] - ETA: 0s - loss: 0.6909 - accuracy: 0.5342
Epoch 70: saving model to Checkpoints/weights.70

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6967 - accuracy: 0.6393 - 195ms/epoch - 729us/step
Loss: 0.6966889500617981, Accuracy: 0.6393002867698669


### Adding more hidden layers


In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30
nodes_hidden_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                3520      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 10)                310       
                                                                 
 dense_12 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [27]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("Checkpoints/",exist_ok=True)
Checkpoint_path = "Checkpoints/weights.{epoch:02d}.hdf5"
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=Checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch", period=5)

In [28]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 779us/step - loss: 17620.7578 - accuracy: 0.4962
Epoch 2/100
804/804 [==============================] - 1s 776us/step - loss: 6682.9194 - accuracy: 0.4962
Epoch 3/100
804/804 [==============================] - 1s 747us/step - loss: 8961.0010 - accuracy: 0.4876
Epoch 4/100
804/804 [==============================] - 1s 752us/step - loss: 4637.0732 - accuracy: 0.5121
Epoch 5/100
800/804 [============================>.] - ETA: 0s - loss: 2084.3887 - accuracy: 0.4973
Epoch 5: saving model to Checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 770us/step - loss: 2078.0818 - accuracy: 0.4971
Epoch 6/100
804/804 [==============================] - 1s 749us/step - loss: 13030.4873 - accuracy: 0.5055
Epoch 7/100
804/804 [==============================] - 1s 763us/step - loss: 219.7436 - accuracy: 0.4977
Epoch 8/100
804/804 [==============================] - 1s 773us/step - loss: 281.7690 - accuracy: 0.5107
Epoch 9/100

804/804 [==============================] - 1s 740us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 64/100
804/804 [==============================] - 1s 751us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 65/100
765/804 [===========================>..] - ETA: 0s - loss: 0.6912 - accuracy: 0.5319
Epoch 65: saving model to Checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 802us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 66/100
804/804 [==============================] - 1s 807us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 67/100
804/804 [==============================] - 1s 760us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 68/100
804/804 [==============================] - 1s 758us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 69/100
804/804 [==============================] - 1s 755us/step - loss: 0.6909 - accuracy: 0.5335
Epoch 70/100
786/804 [============================>.] - ETA: 0s - loss: 0.6909 - accuracy: 0.5340
Epoch 70: saving model to Checkpoints/weights.70

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6880 - accuracy: 0.5290 - 196ms/epoch - 733us/step
Loss: 0.6880440711975098, Accuracy: 0.5289795994758606


### Using different activation functions for the hidden layers

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 80
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="tanh", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 80)                3520      
                                                                 
 dense_14 (Dense)            (None, 30)                2430      
                                                                 
 dense_15 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [31]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs("Checkpoints/",exist_ok=True)
Checkpoint_path = "Checkpoints/weights.{epoch:02d}.hdf5"
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=Checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch", period=5)

In [32]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 718us/step - loss: 0.6943 - accuracy: 0.5238
Epoch 2/100
804/804 [==============================] - 1s 723us/step - loss: 0.6917 - accuracy: 0.5318
Epoch 3/100
804/804 [==============================] - 1s 725us/step - loss: 0.6916 - accuracy: 0.5292
Epoch 4/100
804/804 [==============================] - 1s 693us/step - loss: 0.6916 - accuracy: 0.5324
Epoch 5/100
791/804 [============================>.] - ETA: 0s - loss: 0.6916 - accuracy: 0.5336
Epoch 5: saving model to Checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 714us/step - loss: 0.6916 - accuracy: 0.5335
Epoch 6/100
804/804 [==============================] - 1s 703us/step - loss: 0.6916 - accuracy: 0.5335
Epoch 7/100
804/804 [==============================] - 1s 693us/step - loss: 0.6913 - accuracy: 0.5318
Epoch 8/100
804/804 [==============================] - 1s 705us/step - loss: 0.6915 - accuracy: 0.5299
Epoch 9/100
804/804 [=================

804/804 [==============================] - 1s 709us/step - loss: 0.6916 - accuracy: 0.5308
Epoch 64/100
804/804 [==============================] - 1s 690us/step - loss: 0.6915 - accuracy: 0.5321
Epoch 65/100
789/804 [============================>.] - ETA: 0s - loss: 0.6918 - accuracy: 0.5302
Epoch 65: saving model to Checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 716us/step - loss: 0.6917 - accuracy: 0.5308
Epoch 66/100
804/804 [==============================] - 1s 686us/step - loss: 0.6916 - accuracy: 0.5322
Epoch 67/100
804/804 [==============================] - 1s 695us/step - loss: 0.6919 - accuracy: 0.5316
Epoch 68/100
804/804 [==============================] - 1s 684us/step - loss: 0.6916 - accuracy: 0.5296
Epoch 69/100
804/804 [==============================] - 1s 683us/step - loss: 0.6913 - accuracy: 0.5302
Epoch 70/100
803/804 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5324
Epoch 70: saving model to Checkpoints/weights.70

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6683 - accuracy: 0.5768 - 215ms/epoch - 803us/step
Loss: 0.6682865619659424, Accuracy: 0.5767930150032043


In [34]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")